### This document serves to pull data from repository, format them for our index, and add external variables as needed. 

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sn 
import os 
import geopandas
import matplotlib.pyplot as plt

In [2]:
# list available data source folders
main_folder = 'USDA-AMS-Data-and-Metrics'
folders = os.listdir(main_folder)
print(folders)

['Business Development and Infrastructure', 'Food Access', 'Grants', 'Indicators of Community Wealth', 'Labor', 'Local Food Sales', 'metadata_all_files.csv', 'Point level data', 'README.md']


In [4]:
metadata = pd.read_csv(main_folder + "/" +  'metadata_all_files.csv')

In [46]:
metadata[metadata["variable_name"] == "rwj_food_environment_index"]["variable_definition"]

258    Index of factors that contribute to a healthy ...
Name: variable_definition, dtype: object

In [44]:
metadata[metadata["topic_area"] == metadata["topic_area"].unique()[23]]

,2 pager title,category,topic_area,variable_name,user_friendly_variable_name,variable_definition,periodicity,aggregation_method,format,source,url,last_update_date,general_comments,Libby notes
338,Labor,Labor,Employment,annual_avg_emplvl_NAICS 111 Crop production,"Employment level, annual average, NAICS 111 Cr...",Annual average of monthly employment levels fo...,Yearly,Average,integer,"U.S. Bureau of Labor Statistics, Quarterly Cen...",https://www.bls.gov/cew/downloadable-data-file...,9/23/22,"Annual averages by Industry, privately held bu...",Changed definition
339,Labor,Labor,Employment,annual_avg_emplvl_NAICS 112 Animal production ...,"Employment level, annual average, NAICS 112 An...",Annual average of monthly employment levels fo...,Yearly,Average,integer,"U.S. Bureau of Labor Statistics, Quarterly Cen...",https://www.bls.gov/cew/downloadable-data-file...,9/23/22,"Annual averages by Industry, privately held bu...",Changed definition
340,Labor,Labor,Employment,"annual_avg_emplvl_NAICS 114 Fishing, hunting a...","Employment level, annual average, NAICS 114 Fi...",Annual average of monthly employment levels fo...,Yearly,Average,integer,"U.S. Bureau of Labor Statistics, Quarterly Cen...",https://www.bls.gov/cew/downloadable-data-file...,9/23/22,"Annual averages by Industry, privately held bu...",Changed definition
341,Labor,Labor,Employment,annual_avg_emplvl_NAICS 115 Agriculture and fo...,"Employment level, annual average, NAICS 115 Ag...",Annual average of monthly employment levels fo...,Yearly,Average,integer,"U.S. Bureau of Labor Statistics, Quarterly Cen...",https://www.bls.gov/cew/downloadable-data-file...,9/23/22,"Annual averages by Industry, privately held bu...",Changed definition
342,Labor,Labor,Employment,annual_avg_emplvl_NAICS 311 Food manufacturing,"Employment level, annual average, NAICS 311 Fo...",Annual average of monthly employment levels fo...,Yearly,Average,integer,"U.S. Bureau of Labor Statistics, Quarterly Cen...",https://www.bls.gov/cew/downloadable-data-file...,9/23/22,"Annual averages by Industry, privately held bu...",Changed definition
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,Labor,Labor,Employment,food_preparation_and_serving_related_occupatio...,"Food preparation, serving occupations, Female",Number of females in service occupations of fo...,Yearly,Count,integer,U.S.Census Bureau American Community Survey Ta...,https://data.census.gov/cedsci/table?q=B24010,9/23/22,NaN,Changed variable name
495,Labor,Labor,Employment,farming_fishing_and_forestry_occupations_female,"Farming, fishing, and forestry occupations, Fe...",Number of males in farming fishing and foresty...,Yearly,Count,integer,U.S.Census Bureau American Community Survey Ta...,https://data.census.gov/cedsci/table?q=B24010,9/23/22,NaN,Changed variable name
502,Labor,Labor,Employment,hired_labor_number_workers,"Labor, hired agricultural workers",Number of hired farm labor workers,Yearly,Count,integer,"U.S. Department of Agriculture, National Agric...",https://www.nass.usda.gov/Publications/AgCensu...,9/23/22,All data downloaded from link near the bottom ...,Changed variable name and definition
503,Labor,Labor,Employment,migrant_labor_number_workers,"Labor, agricultural migrant workers",Number of migrant workers,Yearly,Count,integer,"U.S. Department of Agriculture, National Agric...",https://www.nass.usda.gov/Publications/AgCensu...,9/23/22,All data downloaded from link near the bottom ...,Changed variable name and definition


In [5]:
unemployment_income_data = pd.read_excel("Unemployment.xlsx", header=4, index_col=0)
unemployment_income_data.index = [str(fips).zfill(5) for fips in unemployment_income_data.index]
unemployment_income_data = unemployment_income_data.dropna()

In [7]:
external_vars_df = pd.DataFrame()

external_vars_df["income"] = unemployment_income_data["Median_Household_Income_2021"]
external_vars_df["unemployment"] = unemployment_income_data["Unemployment_rate_2022"]

In [12]:
# for now, choose one file to explore. We will start with the first folder, business development and infrastructure.
# Use os to see the file name and pandas to pull the file into the notebook. 
dataframes = []
for folder in os.listdir(main_folder):
    if "." not in folder:
        subfolder = os.path.join(main_folder, folder)
        # Iterate through files within each subfolder
        for file in os.listdir(subfolder):
            if file.endswith(".csv"):
                file_path = os.path.join(subfolder, file)
                # Read CSV file into a DataFrame
                try: 
                    df = pd.read_csv(file_path)
                    df.set_index('fips', inplace=True)
                except: 
                    print(file_path)
                    pass
                
                # Set FIPS code as the index of the DataFrame
                # Append DataFrame to the list
                dataframes.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes)


In [13]:
#fill leading zeroes in fips codes
combined_df.index = combined_df.index.astype(str).str.zfill(5)

In [14]:
combined_df

,county_name,state_name,category,topic_area,year,variable_name,value,lat,long,org_type
fips,,,,,,,,,,
04013,Maricopa County,Arizona,Processing & Distribution,Food hubs,2022,number_food_hubs,1.0,NaN,NaN,NaN
06057,Nevada County,California,Processing & Distribution,Food hubs,2022,number_food_hubs,1.0,NaN,NaN,NaN
06073,San Diego County,California,Processing & Distribution,Food hubs,2022,number_food_hubs,1.0,NaN,NaN,NaN
06075,San Francisco County,California,Processing & Distribution,Food hubs,2022,number_food_hubs,1.0,NaN,NaN,NaN
08003,Alamosa County,Colorado,Processing & Distribution,Food hubs,2022,number_food_hubs,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
32003,Clark County,Nevada,NaN,NaN,NaN,location_colleges_universities,NaN,36.142355,-115.206409,Colleges and universities
32003,Clark County,Nevada,NaN,NaN,NaN,location_colleges_universities,NaN,36.117236,-115.158173,Colleges and universities
34007,Camden County,New Jersey,NaN,NaN,NaN,location_colleges_universities,NaN,39.939428,-75.015417,Colleges and universities


In [84]:
# Get unique variable names
variables = list(combined_df["variable_name"].unique())

# Create a new DataFrame with variable names as row headers
counties = combined_df.index.unique()

column_data = pd.DataFrame(index = counties, columns = variables)

for var in variables: 
    # subset containing all abs with selected variable name
    sub_data = combined_df[combined_df["variable_name"] == var]  
    # keeps only last obs if data are recorded for multiple years
    sub_data = sub_data[~sub_data.index.duplicated(keep="last")]  
    #record value or if there is no value Nan
    for county in counties: 
        if county in sub_data.index:  
                column_data[var].loc[county] = sub_data["value"].loc[county]
        else: 
                column_data[var].loc[county] = np.nan

In [85]:
column_data

,number_food_hubs,number_meat_processors,number_colleges_universities,serve_local_food,salad_bar,local_salad_bar,local_food_cost,school_garden,work_inside_state_out_of_county,work_outside_state,...,d2c_only,intermediated_only,d2c_intermediated,local,nonlocal,d2c_only_p,intermediated_only_p,d2c_intermediated_p,local_p,nonlocal_p
fips,,,,,,,,,,,,,,,,,,,,,
04013,1.0,18.0,75.0,64.0,104.0,35.0,15707505.0,9.0,1.0,1.0,...,148.0,28.0,16.0,192.0,1682.0,0.078975,0.014941,0.008538,0.102455,0.897545
06057,1.0,NaN,1.0,4.0,1.0,1.0,127500.0,3.0,22.7,2.6,...,113.0,27.0,27.0,167.0,506.0,0.167905,0.040119,0.040119,0.248143,0.751857
06073,1.0,27.0,68.0,29.0,34.0,25.0,9358163.0,12.0,1.9,0.6,...,673.0,354.0,87.0,1114.0,3968.0,0.132428,0.069658,0.017119,0.219205,0.780795
06075,1.0,7.0,21.0,2.0,NaN,NaN,3001500.0,1.0,20.8,0.5,...,0.0,4.0,0.0,4.0,6.0,0.0,0.4,0.0,0.4,0.6
08003,1.0,1.0,1.0,2.0,2.0,2.0,30000.0,NaN,19.1,1.4,...,15.0,6.0,2.0,23.0,257.0,0.053571,0.021429,0.007143,0.082143,0.917857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
# add unemployment and median income to dataset
for var in external_vars_df: 
    column_data[var] = external_vars_df[var].reindex_like(column_data)
    variables.append(var)


In [87]:
# select only columns with greater than half (or whatever cutoff) of entries filled
abundant_vars = []
required_proportion = 0
criteria = 3247 * required_proportion   
for var in variables: 
    if len(column_data[var][~column_data[var].isna()]) > criteria: 
        abundant_vars.append(var)
    else: 
        continue 

In [88]:
final_data = column_data[abundant_vars]

In [89]:
final_data = final_data.astype(np.float64)
final_data.to_csv("full_column_data.csv")

In [95]:
final_data

,number_food_hubs,number_meat_processors,number_colleges_universities,serve_local_food,salad_bar,local_salad_bar,local_food_cost,school_garden,work_inside_state_out_of_county,work_outside_state,...,d2c_intermediated,local,nonlocal,d2c_only_p,intermediated_only_p,d2c_intermediated_p,local_p,nonlocal_p,income,unemployment
fips,,,,,,,,,,,,,,,,,,,,,
04013,1.0,18.0,75.0,64.0,104.0,35.0,15707505.0,9.0,1.0,1.0,...,16.0,192.0,1682.0,0.078975,0.014941,0.008538,0.102455,0.897545,76230.0,3.3
06057,1.0,NaN,1.0,4.0,1.0,1.0,127500.0,3.0,22.7,2.6,...,27.0,167.0,506.0,0.167905,0.040119,0.040119,0.248143,0.751857,77220.0,3.4
06073,1.0,27.0,68.0,29.0,34.0,25.0,9358163.0,12.0,1.9,0.6,...,87.0,1114.0,3968.0,0.132428,0.069658,0.017119,0.219205,0.780795,90756.0,3.4
06075,1.0,7.0,21.0,2.0,NaN,NaN,3001500.0,1.0,20.8,0.5,...,0.0,4.0,6.0,0.000000,0.400000,0.000000,0.400000,0.600000,119776.0,2.5
08003,1.0,1.0,1.0,2.0,2.0,2.0,30000.0,NaN,19.1,1.4,...,2.0,23.0,257.0,0.053571,0.021429,0.007143,0.082143,0.917857,51009.0,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
final_data.describe()

,number_food_hubs,number_meat_processors,number_colleges_universities,serve_local_food,salad_bar,local_salad_bar,local_food_cost,school_garden,work_inside_state_out_of_county,work_outside_state,...,d2c_intermediated,local,nonlocal,d2c_only_p,intermediated_only_p,d2c_intermediated_p,local_p,nonlocal_p,income,unemployment
count,23.0,1556.000000,1461.000000,2339.000000,2221.000000,1560.000000,2.212000e+03,971.000000,3222.000000,3222.000000,...,3076.000000,3076.000000,3076.000000,3076.000000,3076.000000,3076.000000,3076.000000,3076.000000,3128.000000,3128.000000
mean,1.0,3.753213,4.302533,3.044036,2.947771,2.389103,4.408139e+05,1.723996,31.156176,4.382309,...,5.228544,46.461313,617.444408,0.057910,0.005922,0.009235,0.073068,0.926932,58924.179348,3.588587
std,0.0,10.156690,9.227204,4.371366,4.122665,2.897303,1.736248e+06,1.488800,18.216651,7.562592,...,10.317628,67.264285,501.954085,0.054981,0.013182,0.019105,0.072604,0.072604,15283.582415,1.226107
min,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,0.000000,0.000000,...,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.333333,25653.000000,0.600000
25%,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.970500e+04,1.000000,15.800000,0.700000,...,0.000000,11.000000,292.000000,0.021057,0.000000,0.000000,0.025995,0.906166,49002.000000,2.700000
50%,1.0,2.000000,2.000000,2.000000,2.000000,1.000000,7.500000e+04,1.000000,29.800000,1.400000,...,2.000000,26.000000,501.000000,0.042892,0.002015,0.003241,0.051167,0.948833,56626.000000,3.400000
75%,1.0,3.000000,4.000000,3.000000,3.000000,3.000000,2.507750e+05,2.000000,44.800000,4.400000,...,6.000000,57.000000,801.250000,0.077596,0.006494,0.009777,0.093834,0.974005,65643.000000,4.200000
max,1.0,260.000000,205.000000,69.000000,104.000000,44.000000,4.106969e+07,13.000000,86.900000,55.800000,...,196.000000,1114.000000,5487.000000,0.666667,0.400000,0.312500,0.666667,1.000000,153716.000000,14.700000
